In [11]:
import json
from pathlib import Path
import pandas as pd 

In [18]:
def extract_experiments(filepath: Path):
    with open(filepath) as f:
        data = json.load(f)
    
    for item in data:
        if item['name'] != 'master':
            continue

        for experiment in item['experiments']:
            for rev in experiment.get("revs", []):
                params = next(iter(rev['data'].get('params', {}).values()), {}).get("data")
                metrics = next(iter(rev['data'].get('metrics', {}).values()), {}).get("data")
                yield {
                    'name': rev['name'],
                    'params': params,
                    'metrics': metrics,
                }

In [27]:
experiments = list(extract_experiments(Path('dvc-experiments-export.json')))
experiments[0]

{'name': 'cte-3',
 'params': {'prompt': 'cte',
  'model': 'gpt-3.5-turbo',
  'temperature': 0.1,
  'run': 3},
 'metrics': {'exact_match': 0.58,
  'f1': 0.718793650793651,
  'fuzzy_match': 0.66}}

In [30]:
df = pd.json_normalize(experiments).sort_values('name')
df

,name,params.prompt,params.model,params.temperature,params.run,metrics.exact_match,metrics.f1,metrics.fuzzy_match
2,cte-1,cte,gpt-3.5-turbo,0.1,1,0.54,0.673749,0.61
1,cte-2,cte,gpt-3.5-turbo,0.1,2,0.58,0.711287,0.64
0,cte-3,cte,gpt-3.5-turbo,0.1,3,0.58,0.718794,0.66
5,standard-1,standard,gpt-3.5-turbo,0.1,1,0.49,0.608839,0.56
4,standard-2,standard,gpt-3.5-turbo,0.1,2,0.46,0.561866,0.53
3,standard-3,standard,gpt-3.5-turbo,0.1,3,0.48,0.579714,0.55
